In [1]:
import sys
sys.path.append('..')

from grobid_client.grobid_client import GrobidClient
from utils.tei_extraction import extract_abstract, tei_to_full_raw_text
from utils.grobid_service import GrobidService
import subprocess

import joblib
import pandas as pd
import json
import os

import requests
from pathlib import Path
from urllib.parse import urlparse
from bs4 import BeautifulSoup

ImportError: cannot import name 'ApiClient' from 'grobid_client.client' (C:\Users\Profesor\miniconda3\envs\gap-kge\lib\site-packages\grobid_client\client.py)

In [2]:
PROJECT_ROOT = Path.cwd().parent 


GROBID_CONFIG = PROJECT_ROOT / "Grobid" / "config.json"
RSEF_JSON = PROJECT_ROOT / "pipeline" / "rsef_output" / "url_search_output.json"

print(f"Raíz del proyecto: {PROJECT_ROOT}")
print(f"Config de Grobid: {GROBID_CONFIG}")


Raíz del proyecto: c:\Users\M\Desktop\Curro\Proyecto\GAP-KGE
Config de Grobid: c:\Users\M\Desktop\Curro\Proyecto\GAP-KGE\Grobid\config.json


## Paso 1

In [5]:
#pdf_url = "https://aclanthology.org/K18-1051.pdf" # Input by user
pdf_url = "https://aclanthology.org/P18-1012.pdf"
#pdf_url = "https://arxiv.org/pdf/1906.01195v1.pdf"

In [4]:
def download_pdf(pdf_url: str, filename: str = None):
    base_dir = Path.cwd()
    dest_folder = base_dir / "temp"
    dest_folder.mkdir(parents=True, exist_ok=True)

    filename = "pdf_to_convert.pdf"

    final_path = dest_folder / filename

    print(f"Descargando en: {final_path} ...")

    try:
        response = requests.get(pdf_url, stream=True)
        response.raise_for_status()

        with open(final_path, 'wb') as pdf_file:
            for chunk in response.iter_content(chunk_size=8192):
                if chunk:
                    pdf_file.write(chunk)
        
        print("Download succesful.")
        return str(final_path)

    except requests.exceptions.RequestException as e:
        print(f"Something went wrong when downloading the PDF: {e}")
        return None


In [6]:
# pdf_path = download_pdf(pdf_url)

pdf_path = str(Path.cwd() / "rsef_output" / "PDFs" / "pdf_to_convert.pdf")

## Paso 2

In [ ]:
def execute_rsef_command(download_url: str = pdf_url, output_path: str = "rsef_output/"):
    rsef_command = ["rsef", "assess", "-i", download_url, "-o", output_path, "-U"]
    print(f"Executing command: {' '.join(rsef_command)}")

    try:
        result = subprocess.run(rsef_command, capture_output=True, text=True, check=True)
        print("Done!")
        return result.stdout
    
    except subprocess.CalledProcessError as e:
        print("Error when executing rsef.")
        print("Details:", e.stderr)
        return None
    except FileNotFoundError:
        print("No comand rsef found.")
        return None

execute_rsef_command()

Executing command: rsef assess -i https://aclanthology.org/P18-1012.pdf -o rsef_output/ -U
Done!


''

In [21]:
with open(RSEF_JSON, "r", encoding="utf-8") as j:
    rsef_json = json.load(j)

paper_title = rsef_json['RSEF Output'][0]['title']
print(f"Extracted title: {paper_title}")
doi = rsef_json['RSEF Output'][0]['doi']
print(f"Extracted DOI: {doi}")
publication_date = rsef_json['RSEF Output'][0]['publication_date']
print(f"Extracted publication date: {publication_date}")


if rsef_json['RSEF Output'][0]['implementation_urls']:
    implementation = rsef_json['RSEF Output'][0]['implementation_urls'][0]['identifier']
else:
    implementation = "No implementation URL found."
print(f"Extracted implementation URL: {implementation}")

Extracted title: Towards Understanding the Geometry of Knowledge Graph Embeddings
Extracted DOI: 10.18653/v1/p18-1012
Extracted publication date: 2018-01-01
Extracted implementation URL: https://github.com/malllabiisc/kg-geometry


In [23]:
urls_data = rsef_json['RSEF Output'][0]["implementation_urls"]
implementation_list = [item["identifier"] for item in urls_data if "identifier" in item]
print(implementation_list)

['https://github.com/malllabiisc/kg-geometry', 'https://github.com/malllabiisc/kg-geometry']


In [24]:
grobid = GrobidService(config_path = GROBID_CONFIG)


print(f"Extracted title: {paper_title}")
authors = grobid.extract_authors_from_pdf(pdf_path)
print(f"Extracted authors: {authors}")
tei_text = grobid.process_full_text(pdf_path)
abstract = extract_abstract(tei_text)
raw_text = tei_to_full_raw_text(tei_text)

INFO - Loading configuration file from c:\Users\M\Desktop\Curro\Proyecto\GAP-KGE\Grobid\config.json
INFO - Configuration file loaded successfully


Extracted title: Towards Understanding the Geometry of Knowledge Graph Embeddings
Extracted authors: ['Aditya Sharma', 'Partha Talukdar']


In [25]:
model = joblib.load('classification_model\clasificador_textos_v1.pkl')
le = joblib.load('classification_model\label_encoder.pkl')
print("Model and Label Encoder loaded successfully.")

class_numeric_prediction = model.predict([raw_text])
class_prediction = le.inverse_transform(class_numeric_prediction)

Model and Label Encoder loaded successfully.


In [ ]:
# Añadr tareas (SciREX)


In [ ]:
# Extracción de datasets

🔄 Enviando ..\Pruebas\pdfs_prueba\Binarized Knowledge Graph Embeddings.pdf a Datastet...

✅ ¡Extracción completada!
📂 Resultados guardados en 'datasets_output.json'

Resumen del JSON (primeros 500 caracteres):
{'application': 'datastet', 'version': '0.8.1', 'date': '2026-01-21T15:22+0000', 'md5': 'CF9E94BB130F2E320F8034D17EE5CB04', 'pages': [{'page_height': 792.0, 'page_width': 612.0}, {'page_height': 792.0, 'page_width': 612.0}, {'page_height': 792.0, 'page_width': 612.0}, {'page_height': 792.0, 'page_width': 612.0}, {'page_height': 792.0, 'page_width': 612.0}, {'page_height': 792.0, 'page_width': 612.0}, {'page_height': 792.0, 'page_width': 612.0}, {'page_height': 792.0, 'page_width': 612.0}, {'page...


## Paso 3

In [26]:
json_structure = {
    "paper_title": paper_title,
    "paper_url": pdf_url,
    "class": class_prediction[0],
    "authors": authors,
    "abstract": abstract,
    "implementation_url": implementation_list,
    "publication_date": publication_date,
    "paper_doi": doi
}

generated_json_path = "generatedJSON"
json_file_name = "test.json"

complete_path = os.path.join(generated_json_path, json_file_name)
os.makedirs(generated_json_path, exist_ok=True)

with open(complete_path, 'w', encoding='utf-8') as file:
    json.dump(
        json_structure, 
        file, 
        indent=4,
        ensure_ascii=False
    )
print(f"JSON succesfully saved in {complete_path}")

JSON succesfully saved in generatedJSON\test.json


## Pipeline general:
1. Descargar el archivo PDF desde la URL aportada por el usuario
2. Extraer título, autores, abstract y predecir su clase (tareas y código en un futuro próximo)
3. Generar la plantilla (JSON) con la información obtenida

Entiendo que en un futuro estos tres pasos serán métodos que se ejecutarán secuencialmente




Pipeline de descarga de RSEF:
pdf_download_pipeline -> doi_to_downloaded_pdf -> download_pdf_link